In [10]:
import pandas as pd
rides = pd.read_csv('../data/transformed/validated_rides_2024.csv', index_col=0)
rides.head(10)

,pickup_datetime,pickup_latitude,pickup_longitude
0,01/19/2024 05:00:00 PM,41.979071,-87.903040
1,01/28/2024 02:30:00 PM,41.944227,-87.655998
2,01/05/2024 09:00:00 AM,41.944227,-87.655998
3,01/22/2024 08:45:00 AM,41.980264,-87.913625
4,01/18/2024 07:15:00 PM,41.880994,-87.632746
5,01/09/2024 05:00:00 PM,41.965812,-87.655879
6,01/04/2024 07:15:00 AM,41.879255,-87.642649
7,01/25/2024 11:15:00 AM,41.884987,-87.620993
8,01/09/2024 03:15:00 PM,41.859350,-87.617358
9,01/20/2024 04:00:00 PM,41.979071,-87.903040


In [14]:
rides['pickup_datetime'] = pd.to_datetime(rides['pickup_datetime'], format='%m/%d/%Y %I:%M:%S %p')
rides['pickup_hour'] = rides['pickup_datetime'].dt.round('h')


In [15]:
rides.head()

,pickup_datetime,pickup_latitude,pickup_longitude,pickup_hour
0,2024-01-19 17:00:00,41.979071,-87.903040,2024-01-19 17:00:00
1,2024-01-28 14:30:00,41.944227,-87.655998,2024-01-28 14:00:00
2,2024-01-05 09:00:00,41.944227,-87.655998,2024-01-05 09:00:00
3,2024-01-22 08:45:00,41.980264,-87.913625,2024-01-22 09:00:00
4,2024-01-18 19:15:00,41.880994,-87.632746,2024-01-18 19:00:00


In [ ]:
rides = rides[rides['pickup_latitude'].isnull()==False]

,pickup_datetime,pickup_latitude,pickup_longitude,pickup_hour
0,2024-01-19 17:00:00,41.979071,-87.903040,2024-01-19 17:00:00
1,2024-01-28 14:30:00,41.944227,-87.655998,2024-01-28 14:00:00
2,2024-01-05 09:00:00,41.944227,-87.655998,2024-01-05 09:00:00
3,2024-01-22 08:45:00,41.980264,-87.913625,2024-01-22 09:00:00
4,2024-01-18 19:15:00,41.880994,-87.632746,2024-01-18 19:00:00
...,...,...,...,...
6905282,2024-12-31 16:45:00,41.980264,-87.913625,2024-12-31 17:00:00
6905283,2024-12-31 11:30:00,41.899602,-87.633308,2024-12-31 12:00:00
6905284,2024-12-31 15:15:00,41.954028,-87.763399,2024-12-31 15:00:00
6905286,2024-12-31 14:00:00,41.979071,-87.903040,2024-12-31 14:00:00


In [27]:
import pandas as pd
import geopandas as gpd
from shapely.wkt import loads

chicago_zones = pd.read_csv('../data/chicago_geo_data.csv')
chicago_zones["geometry"] = chicago_zones["the_geom"].apply(loads)
chicago_zones = gpd.GeoDataFrame(chicago_zones, geometry="geometry", crs="EPSG:4326")

# Verificar que las geometrías sean correctas
print(chicago_zones.head())


                                            the_geom           PRI_NEIGH  \
0  MULTIPOLYGON (((-87.60670812560372 41.81681377...     Grand Boulevard   
1  MULTIPOLYGON (((-87.62760697485348 41.87437097...        Printers Row   
2  MULTIPOLYGON (((-87.66706868914602 41.88885187...       United Center   
3  MULTIPOLYGON (((-87.65833494805533 41.92166144...  Sheffield & DePaul   
4  MULTIPOLYGON (((-87.74059567509266 41.88782316...       Humboldt Park   

            SEC_NEIGH    SHAPE_AREA     SHAPE_LEN  \
0         BRONZEVILLE  4.849250e+07  28196.837157   
1        PRINTERS ROW  2.162138e+06   6864.247156   
2       UNITED CENTER  3.252051e+07  23101.363745   
3  SHEFFIELD & DEPAUL  1.048259e+07  13227.049745   
4       HUMBOLDT PARK  1.250104e+08  46126.751351   

                                            geometry  
0  MULTIPOLYGON (((-87.60671 41.81681, -87.6067 4...  
1  MULTIPOLYGON (((-87.62761 41.87437, -87.6276 4...  
2  MULTIPOLYGON (((-87.66707 41.88885, -87.66707 ...  
3  M

In [34]:
from shapely.geometry import Point

# Convertir el DataFrame de taxis en un GeoDataFrame
geometry = [Point(xy) for xy in zip(rides['pickup_longitude'], rides['pickup_latitude'])]
rides_gdf = gpd.GeoDataFrame(rides, geometry=geometry, crs="EPSG:4326")

# Hacer el join espacial para asignar cada taxi a un barrio
rides_with_zones = gpd.sjoin(rides_gdf, chicago_zones[['PRI_NEIGH', 'geometry']], 
                             how="left", predicate="within")

# Mantener solo las columnas originales de rides + el barrio asignado
rides_with_zones = rides_with_zones[rides.columns.tolist() + ['PRI_NEIGH']]

# Rellenar valores nulos con "Outside Chicago"
rides_with_zones['neighborhood_name'] = rides_with_zones['PRI_NEIGH'].fillna("Outside Chicago")
rides_with_zones.drop(columns='PRI_NEIGH', inplace=True)
# Ver los primeros resultados
print(rides_with_zones.head())


      pickup_datetime  pickup_latitude  pickup_longitude         pickup_hour  \
0 2024-01-19 17:00:00        41.979071        -87.903040 2024-01-19 17:00:00   
1 2024-01-28 14:30:00        41.944227        -87.655998 2024-01-28 14:00:00   
2 2024-01-05 09:00:00        41.944227        -87.655998 2024-01-05 09:00:00   
3 2024-01-22 08:45:00        41.980264        -87.913625 2024-01-22 09:00:00   
4 2024-01-18 19:15:00        41.880994        -87.632746 2024-01-18 19:00:00   

  neighborhood_name  
0            O'Hare  
1         Lake View  
2         Lake View  
3            O'Hare  
4              Loop  


In [35]:
rides_with_zones

,pickup_datetime,pickup_latitude,pickup_longitude,pickup_hour,neighborhood_name
0,2024-01-19 17:00:00,41.979071,-87.903040,2024-01-19 17:00:00,O'Hare
1,2024-01-28 14:30:00,41.944227,-87.655998,2024-01-28 14:00:00,Lake View
2,2024-01-05 09:00:00,41.944227,-87.655998,2024-01-05 09:00:00,Lake View
3,2024-01-22 08:45:00,41.980264,-87.913625,2024-01-22 09:00:00,O'Hare
4,2024-01-18 19:15:00,41.880994,-87.632746,2024-01-18 19:00:00,Loop
...,...,...,...,...,...
6905283,2024-12-31 11:30:00,41.899602,-87.633308,2024-12-31 12:00:00,River North
6905284,2024-12-31 15:15:00,41.954028,-87.763399,2024-12-31 15:00:00,Portage Park
6905285,2024-12-31 10:45:00,NaN,NaN,2024-12-31 11:00:00,Outside Chicago
6905286,2024-12-31 14:00:00,41.979071,-87.903040,2024-12-31 14:00:00,O'Hare


In [45]:
agg_rides = rides_with_zones.groupby(['pickup_hour', 'neighborhood_name']).size().reset_index()
agg_rides.rename(columns={0:'rides'}, inplace=True)
agg_rides.sort_values(by='rides')

,pickup_hour,neighborhood_name,rides
13,2024-01-01 00:00:00,Hyde Park,1
11,2024-01-01 00:00:00,Grand Boulevard,1
8,2024-01-01 00:00:00,Fuller Park,1
7,2024-01-01 00:00:00,Englewood,1
6,2024-01-01 00:00:00,Edgewater,1
...,...,...,...
92468,2024-03-20 20:00:00,O'Hare,614
22838,2024-01-21 20:00:00,O'Hare,626
167066,2024-05-20 22:00:00,O'Hare,630
406698,2024-12-01 20:00:00,O'Hare,649


In [ ]:
from tqdm import tqdm

def add_missing_slots(agg_rides: pd.DataFrame) -> pd.DataFrame:
    
    location_ids = agg_rides['neighborhood_name'].unique()
    full_range = pd.date_range(
        agg_rides['pickup_hour'].min(), agg_rides['pickup_hour'].max(), freq='H')
    output = pd.DataFrame()
    for location_id in tqdm(location_ids):

        # keep only rides for this 'location_id'
        agg_rides_i = agg_rides.loc[agg_rides.neighborhood_name == location_id, ['pickup_hour', 'rides']]
            
        # quick way to add missing dates with 0 in a Series
        # taken from https://stackoverflow.com/a/19324591
        agg_rides_i.set_index('pickup_hour', inplace=True)
        agg_rides_i.index = pd.DatetimeIndex(agg_rides_i.index)
        agg_rides_i = agg_rides_i.reindex(full_range, fill_value=0)
        
        # add back `location_id` columns
        agg_rides_i['neighborhood_name'] = location_id

        output = pd.concat([output, agg_rides_i])
    
    # move the purchase_day from the index to a dataframe column
    output = output.reset_index().rename(columns={'index': 'pickup_hour'})
    
    return output

In [47]:
agg_rides_all_slots = add_missing_slots(agg_rides)
agg_rides_all_slots

C:\Users\joral_08cedew\AppData\Local\Temp\ipykernel_33040\2205724025.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_range = pd.date_range(
100%|██████████| 95/95 [00:02<00:00, 45.27it/s]


,pickup_hour,rides,neighborhood_name
0,2024-01-01 00:00:00,2,Albany Park
1,2024-01-01 01:00:00,0,Albany Park
2,2024-01-01 02:00:00,0,Albany Park
3,2024-01-01 03:00:00,3,Albany Park
4,2024-01-01 04:00:00,0,Albany Park
...,...,...,...
905250,2025-01-31 20:00:00,0,Andersonville
905251,2025-01-31 21:00:00,0,Andersonville
905252,2025-01-31 22:00:00,0,Andersonville
905253,2025-01-31 23:00:00,0,Andersonville


In [ ]:
agg_rides_all_slots.neighborhood_name.unique()

array(['Albany Park', 'Archer Heights', 'Ashburn', 'Avondale',
       'Chicago Lawn', 'East Village', 'Edgewater', 'Englewood',
       'Fuller Park', 'Garfield Ridge', 'Gold Coast', 'Grand Boulevard',
       'Humboldt Park', 'Hyde Park', 'Kenwood', 'Lake View',
       'Lincoln Park', 'Lincoln Square', 'Little Italy, UIC',
       'Logan Square', 'Loop', 'Near South Side', 'North Center',
       'Norwood Park', "O'Hare", 'Outside Chicago', 'River North',
       'Rogers Park', 'Rush & Division', 'Sheffield & DePaul',
       'South Shore', 'Streeterville', 'Uptown', 'West Loop', 'West Town',
       'Chatham', 'Grand Crossing', 'Grant Park', 'Jefferson Park',
       'Lower West Side', 'Pullman', 'West Elsdon', 'West Ridge',
       'Wicker Park', 'Woodlawn', 'Armour Square', 'Austin', 'Boystown',
       'Bridgeport', 'Brighton Park', 'Douglas', 'Hermosa', 'Irving Park',
       'North Park', 'Portage Park', 'Roseland', 'Washington Heights',
       'Wrigleyville', 'Gage Park', 'Museum Campus',

In [56]:
from typing import Optional, List
import pandas as pd
import plotly.express as px

def plot_rides(
    rides: pd.DataFrame,
    locations: Optional[List[int]] = None,
    start_date: Optional[str] = None,
    end_date: Optional[str] = None
):
    """
    Plot time-series data with optional location and date filtering.

    :param rides: DataFrame with ride data.
    :param locations: List of neighborhoods to filter (Optional).
    :param start_date: Start date as 'YYYY-MM-DD' (Optional).
    :param end_date: End date as 'YYYY-MM-DD' (Optional).
    """
    rides_to_plot = rides.copy()
    
    # Filtrar por ubicación si se especifica
    if locations:
        rides_to_plot = rides_to_plot[rides_to_plot.neighborhood_name.isin(locations)]
    
    # Filtrar por rango de fechas si se especifica
    if start_date:
        rides_to_plot = rides_to_plot[rides_to_plot["pickup_hour"] >= start_date]
    if end_date:
        rides_to_plot = rides_to_plot[rides_to_plot["pickup_hour"] <= end_date]

    # Graficar
    fig = px.line(
        rides_to_plot,
        x="pickup_hour",
        y="rides",
        color="neighborhood_name",
        template="none",
    )

    fig.show()


In [67]:
plot_rides(agg_rides_all_slots, locations=["Gold Coast", "Loop", "O'Hare"], start_date="2024-01-01", end_date="2024-02-01")


In [68]:
agg_rides_all_slots.to_csv("../data/transformed/ts_data_2024.csv")